### 파킨슨병 데이터
- 환자들의 뇌를 촬영한 사진의 상태를 기록한 자료에 각 환자의 상태 status(1: 파킨슨병 진단, 0: 파킨슨병 아님)로 추가한 테이블
- (data/parkinsons.csv)
1. 파킨슨 병을 예측하는 모델로 로지스틱 회귀모형을 적용하여 생성
2. 파킨슨병을 예측하는데 영향을 미치는 변수를 중요한 순서대로 3개 선정
3. 파킨슨 병을 진단하는 기준를 함수로 생성하여(매개변수명 = threshold, 함수명 = cutoff)을 0.5로 했을 때와 0.8로 했을 때 F1-스코어를 비교
    - 분석 조건
        - 필요 없는 컬럼 name을 삭제
        - 데이터의 정규화는 min-max 스케일러 사용
        - 로지스틱 회귀를 위한 상수항 추가
        - status는 카테고리 타입으로 변환
        - 트레이닝셋과 테스트셋 비율은 9:1
        - 모델은 로지스틱 회귀분석 사용
        - 모델의 최적화 방법론은 "bfgs" 사용

In [60]:
import pandas as pd 
df= pd.read_csv('../ubion/data/Parkinsson disease.csv')
df.head(1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              195 non-null    object 
 1   MDVP:Fo(Hz)       195 non-null    float64
 2   MDVP:Fhi(Hz)      195 non-null    float64
 3   MDVP:Flo(Hz)      195 non-null    float64
 4   MDVP:Jitter(%)    195 non-null    float64
 5   MDVP:Jitter(Abs)  195 non-null    float64
 6   MDVP:RAP          195 non-null    float64
 7   MDVP:PPQ          195 non-null    float64
 8   Jitter:DDP        195 non-null    float64
 9   MDVP:Shimmer      195 non-null    float64
 10  MDVP:Shimmer(dB)  195 non-null    float64
 11  Shimmer:APQ3      195 non-null    float64
 12  Shimmer:APQ5      195 non-null    float64
 13  MDVP:APQ          195 non-null    float64
 14  Shimmer:DDA       195 non-null    float64
 15  NHR               195 non-null    float64
 16  HNR               195 non-null    float64
 1

In [30]:
x.columns

Index(['MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)',
       'MDVP:Jitter(Abs)', 'MDVP:RAP', 'MDVP:PPQ', 'Jitter:DDP',
       'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5',
       'MDVP:APQ', 'Shimmer:DDA', 'NHR', 'HNR', 'RPDE', 'DFA', 'spread1',
       'spread2', 'D2', 'PPE'],
      dtype='object')

In [105]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
scale = MinMaxScaler()
lg = LogisticRegression(solver='lbfgs')
# 'bfgs'
# 기울기 추정을 위해 이전 기울기 정보를 사용하며, 경사하강법을 쓰고, hessian 행렬 업데이트 방법을 사용
# hessian 행렬 방식은 피쳐가 많은 비선형 문제에서 효과적임. 
# 근데 로지스틱엔 bfgs없으므로 lbfgs사용 
# 두 방법의 차이는 메모리 사용방식의 차이이며, lbfgs가 훨씬 더 메모리 효율적

x=df.drop(['status','name'], axis=1)

x=scale.fit_transform(x)
y=df['status'].astype('category')

x_tr,x_test,y_tr,y_test = train_test_split(x,y, test_size=0.1, stratify=y, random_state=12)

lg1=lg.fit(x_tr,y_tr.ravel())
coef=pd.DataFrame(data=lg1.coef_,columns=df.drop(['status','name'],axis=1).columns)

coef.iloc[0].nlargest(3) #spread1, spread2, PPE 가 핵심 3개
pred=lg.predict_proba(x_test)[:,1] # 1일확률

In [106]:
from sklearn.metrics import f1_score
score=f1_score
def cutoff(threshold):
    result=[]
    for i in range (0,len(pred),1):
        if pred[i]>threshold:
            result.append(1)
        else:
            result.append(0)
    f1_threshold=score(result,y_test)
    
    result2=[]
    for i in range (0,len(pred),1):
        if pred[i]>0.5:
            result2.append(1)
        else:
            result2.append(0)
    f1_half=score(result2,y_test)
    
    
    
    return print(f'설정된 임계점의 f1값:{f1_threshold}\n 임계점0.5일때 f1값:{f1_half}')

In [107]:
cutoff(0.8)

설정된 임계점의 f1값:0.6666666666666667
 임계점0.5일때 f1값:0.9333333333333333
